# Todo - will remove this section before submittion 
* Todo 1
* change code using without for loops

Prof notes - 
Given that everyone going different things for the project I want you to add a short  description in your notebook. There are up to six things that I want.

First if you are working on a team then include the names and Red IDs of both team members at the top of the readme file. Just have one person submit the project so I don’t end up grading the same project twice.

Second a brief description of what the project does. What are the goals of your assignment? Did you succeed? What did you learn. In the assignments I knew what you were to do. So if you just had code and tables I could figure things out. In your project you need to explain what your code is doing and what the output of your code means. In Jupyter notebooks you can use markdown cells to contain text. Use the markdown cells to to explain your code and results as they are easier to read then comments in code. Use the markdown cells to add headers (Introduction, How to Run, Results, etc) to give the notebook some structure.  

Third any special instructions that I need to run the project. 

Fourth if you use any third party libraries list them and give the pip command needed to install them. A sentence or two on what the library does for you. If your idea comes from a site like Kaggle include a link to it. 

Fifth a list know issues with the project. If a feature is not working let me know. If a feature is not working I will grade it more severe than if you do not tell me.

Six do not upload large data files. If your total upload to the course portal is 5 MB you will be fine. If people start uploading large files to the course portal the server will crash causing you and other people in the class to panic. (I need to rewrite that server.) If you data file puts you over that limit add a command in your jupyter notebook that will  download your data file from its source. The following command in a code cell on a Unix machine will download covid data at the given https address to the local file covid.csv.


!curl -o covid.csv https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv 


If your code takes a long time to run on the original data set then I would like a smaller dataset to run while grading. If the file is small enough it can be combined with your notebook and up loaded. If not provide a method to download the file: dropbox link, Google drive link, etc. If you are not able to do this contact me to make other arrangements. Always include a link to access the original data set file.


As with assignments and the exam you will submit the project to the course portal.

#  Big Data CS649 Project -  Kavya and Sachin Kumar (RedIds - 825893660 and 823431551)

## Info

### Objective :
- Analysing the trend of crypto currency for the last few years.
- Cryptocurrency price Prediction using Deep learning. (for example ML models - Linear Regression,  K-Nearest Neighbors.)
- Analysis of social media on the crypto prize. Sentiments using few keywords in the tweets and relate with the currency price.  Positive, negative or neutral emotions.
- Our main focus would be on Twitter and Bit-coin. We can explore more currencies and platforms if time permits. 


### Datasets :
- Tweets dataset : https://www.kaggle.com/alaix14/bitcoin-tweets-20160101-to-20190329
    - Contains all the bitcoin tweets
    - date range is from 2016-Jan-01 to 2019-March-29
    
### Special Library/Packages installed : 
- TextBlob
    - pip install -U textblob
    - https://textblob.readthedocs.io/en/dev/install.html
- Language detector and translator
    - pip install google_trans_new
    - https://github.com/lushan88a/google_trans_new
- pip install pandas-profiling
- conda install graphviz
- #todo : might need to delete below packages
- pip install atoti 
- pip install atoti-aws



### Approach :


### Chalenges :
 - Not able to install few packages on windows like whatthelang
    - Language Prediction
    - https://github.com/indix/whatthelang
    - pip install -r requirements.txt
    - pip install whatthelang
 - Google translator stopped translating the tweets after certain time with below error.
    - Error - "Our systems have detected unusual traffic from your computer network"

In [1]:
# ! pip install -U textblob
# ! pip install google_trans_new

In [2]:
import numpy as np 
import pandas as pd 
from textblob import TextBlob
import matplotlib.pyplot as plt

######### Dataset file paths ###########
bitcoin_tweets_dataset_filepath = '../input_datasets/tweets.csv'
processed_bitcoin_tweets_dataset_filepath = '../input_datasets/tweets_processed.csv'


def read_and_clean_input_file(input_file):
    # todo : using small dataset for the devlopment purpose
    # bitcoin_tweets_dataset_df_raw = pd.read_csv(bitcoin_tweets_dataset_filepath, sep=';')
    bitcoin_tweets_dataset_df_raw = pd.read_csv(input_file, sep=';',nrows=100)

    display(bitcoin_tweets_dataset_df_raw)
    # display(bitcoin_tweets_dataset_df_raw.info())

    bitcoin_tweets_dataset_df_cleaned = bitcoin_tweets_dataset_df_raw.drop(['replies','likes','retweets','url','id','user','fullname'],axis = 1)
    bitcoin_tweets_dataset_df_cleaned.reset_index(drop=True, inplace=True)
    bitcoin_tweets_dataset_df_cleaned['timestamp'] = pd.to_datetime(bitcoin_tweets_dataset_df_cleaned['timestamp'])
    # df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
    return bitcoin_tweets_dataset_df_cleaned

In [3]:
########### Below function is just for anlysing the dataset #####################################################
    
def analysing_dataset(df):

    display(df.info())
#     df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    display(df.info())
    
##########    filtering only one date  ########################################
    temp = df.query("timestamp == ['2019-05-27 11:49:14+00']")
    display(temp)

##########     filtering date range using between  ##############################
#     display(df)
    start_date = '2019-05-27 11:49:14+00'
    end_date = '2019-05-20 12:34:03+00'
    temp = df[df['timestamp'].between(start_date, end_date, inclusive=True)]
    display(temp)
    
    temp = df.loc[df['timestamp'].between(start_date, end_date, inclusive=True)]
    display(temp)

    
##########     filtering date range using loc -- this has good performane ##############################
    df = df.set_index(['timestamp'])
    temp = df.loc[start_date:end_date]
    display(temp)
    

# create a single column DataFrame with dates going from Jan 1st 2018 to Jan 1st 2019
df = pd.DataFrame({'timestamp':pd.date_range('2019-01-01','2019-05-01')})
    
#     test = df['text']
#     display(test)


########### Testing block #####################################################################################################
# bitcoin_tweets_dataset_df_cleaned = read_and_clean_input_file(bitcoin_tweets_dataset_filepath)
# analysing_dataset(bitcoin_tweets_dataset_df_cleaned)

In [4]:
from google_trans_new import google_translator  

translator = google_translator() 
detector = google_translator()  

def add_english_translated_tweets_to_df(df):
    english_translated_col = []
    for tweet in df['text']:
        if len(tweet)!=0:
            detect_lang = detector.detect(tweet)
            if (detect_lang != ['en', 'english']) :
                english_translated_col.append(translator.translate(tweet,lang_tgt='en') )
            else:
                english_translated_col.append(tweet)
        else:
            english_translated_col.append(None)

    df['english_translated'] = english_translated_col
    return df

############  Testing block  ##########################################

# function to detect lan
detector = google_translator()  
text = "È appena uscito un nuovo video! LES CRYPTOMONN..."
# text = "Another text which is already in english, so should not be translated."
detect_lang = detector.detect(text)
print(detect_lang)


# function to translate the text to english
if (detect_lang != ['en', 'english']) :
    print(translator.translate(text,lang_tgt='en') )
    

['it', 'italian']
A new video has just come out! Les Cryptomonn ... 


In [5]:
def read_clean_process_write_input_file(input_file):
    bitcoin_tweets_dataset_df_cleaned = read_and_clean_input_file(input_file)
    display(bitcoin_tweets_dataset_df_cleaned.info())
    display(bitcoin_tweets_dataset_df_cleaned)

    bitcoin_tweets_dataset_df_cleaned = add_english_translated_tweets_to_df(bitcoin_tweets_dataset_df_cleaned)
    display(bitcoin_tweets_dataset_df_cleaned)
    ##### Write the processed ouput to disk. This is to avoid unnecssary english translation everytime we run the program
    processed_bitcoin_tweets_dataset_filepath = '../input_datasets/tweets_processed.csv'
    bitcoin_tweets_dataset_df_cleaned.to_csv(index=False, sep=',' , path_or_buf=processed_bitcoin_tweets_dataset_filepath)
    
############  Testing block  ##########################################  
# read_clean_process_write_input_file(bitcoin_tweets_dataset_filepath)

In [6]:
# todo: Note the seperator delimeter. orginal file has ';' and processed file has ','. 
# Should be good across complete file

import os.path
from os import path
processed_bitcoin_tweets_dataset_filepath 

if path.exists(processed_bitcoin_tweets_dataset_filepath):
    print ("processed bitcoin_tweets dataset filep exist")
else:
    print ("processed bitcoin_tweets dataset file does not exist")
    read_clean_process_write_input_file(bitcoin_tweets_dataset_filepath)


processed_bitcoin_tweets_dataset_df = pd.read_csv(processed_bitcoin_tweets_dataset_filepath, sep=',')
processed_bitcoin_tweets_dataset_df['timestamp'] = pd.to_datetime(processed_bitcoin_tweets_dataset_df['timestamp'])
processed_bitcoin_tweets_dataset_df = processed_bitcoin_tweets_dataset_df.set_index(['timestamp'])
# display(processed_bitcoin_tweets_dataset_df.info())
display(processed_bitcoin_tweets_dataset_df)


processed bitcoin_tweets dataset filep exist


,text,english_translated
timestamp,,
2019-05-27 11:49:14+00:00,È appena uscito un nuovo video! LES CRYPTOMONN...,A new video has just come out! Les Cryptomonna...
2019-05-27 11:49:18+00:00,Cardano: Digitize Currencies; EOS https://t.co...,Cardano: Digitize Currencies; EOS https://t.co...
2019-05-27 11:49:06+00:00,Another Test tweet that wasn't caught in the s...,Another Test tweet that wasn't caught in the s...
2019-05-27 11:49:22+00:00,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
2019-05-27 11:49:23+00:00,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...
...,...,...
2019-05-18 21:37:23+00:00,@xrp_news @coinbase @APompliano I’m ‘fairly’ n...,@xrp_news @coinbase @APompliano I’m ‘fairly’ n...
2019-05-27 00:02:39+00:00,Keep your eye on the prize bears you got this....,Keep your eye on the prize bears you got this....
2019-05-27 11:50:24+00:00,(15 Great Ways for Absolute Beginners to Learn...,(15 Great Ways for Absolute Beginners to Learn...


In [7]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

""" Utility function to clean tweet text by removing special characters and links using regex"""
def clean_special_chars(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+://\S+)", " ", text).split())


""" Utility function to clean tweet text and remove verbose english words using nltk stopwords"""
def clean_tweet(text):
    text = clean_special_chars(text)
    forbidden_words = set(stopwords.words('english'))
    text = ' '.join(text.split('.'))
    text = re.sub('\/',' ',text)
    text = text.strip('\'"')
    text = re.sub(r'@([^\s]+)',r'\1',text)
    text = re.sub(r'\\',' ',text)
    text = text.lower()
    text = re.sub('[\s]+', ' ', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub(r'((http)\S+)','',text)
    text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z]', ' ', text.strip().lower())).strip()
    text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
    text = [word for word in text.split() if word not in forbidden_words]
    return ' '.join(text)


processed_bitcoin_tweets_dataset_df['english_keywords'] = processed_bitcoin_tweets_dataset_df['english_translated'].apply(lambda text: clean_tweet(text))
processed_bitcoin_tweets_dataset_df.drop(columns='english_translated', inplace=True)
processed_bitcoin_tweets_dataset_df.sample(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sk105659\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,english_keywords
timestamp,,
2019-05-27 11:49:38+00:00,"Bitcoin SV (BSV) Price Spikes 33%, Gains $1 Bi...",bitcoin sv bsv price spikes gains billion copy...
2019-05-27 11:49:04+00:00,I didn’t buy enough #bitcoin at thee bottom! \...,buy enough bitcoin thee bottom waiting hoping ...
2019-05-10 09:59:45+00:00,@mark_dow Bitcoin is the one imv,dow bitcoin one imv
2019-05-27 11:49:32+00:00,"CHANGE IS COMING...GET READY!!! Boom, Another ...",change coming get ready boom another cb jab no...
2019-05-27 11:49:48+00:00,"@APompliano If by Bitcoin you mean BSV, I full...",bitcoin mean bsv fully agree however talking b...
2019-05-27 11:50:03+00:00,"ICYMI: Ruden Holding, a Japanese real estate c...",icymi ruden holding japanese real estate compa...
2019-05-27 11:49:25+00:00,#btc inceldiği yerden kopsun bakalım 17:00 ye ...,btc let see movement review expect movement th...
2019-05-27 11:50:09+00:00,this channel called $DGB before it made 6x. th...,channel called dgb made x share high reward lo...
2019-05-27 11:50:06+00:00,為替(19/05/27 20:50㈪現在)\n\n米ドル → 109.49円\n香港ドル →...,exchange current us dollar yen hong kong dolla...


In [8]:
from textblob import TextBlob

def sentiment(text):
    # create TextBlob object of passed tweet text
    text_analysis = TextBlob(text)
    return text_analysis.sentiment.polarity

processed_bitcoin_tweets_dataset_df['sentiment'] = processed_bitcoin_tweets_dataset_df['english_keywords'].apply(lambda text: sentiment(text)) # new column of sentiment
processed_bitcoin_tweets_dataset_df.sample(10)

,text,english_keywords,sentiment
timestamp,,,
2019-05-27 11:49:42+00:00,"Global coin (GC) já começa errada, sendo uma c...",global coin gc already gets wrong coin fight b...,-0.071429
2019-05-27 11:50:02+00:00,dash/btc: 0.0188\ndash/usd: 161.87\nbtc/usd: ...,dash btc dash usd btc usd,0.000000
2019-05-27 11:25:16+00:00,♦️Huobi Wallet Event♦️\n\n⚠️Only 5 Hours left ...,huobi wallet event hours left participate simp...,0.000000
2019-05-27 11:49:40+00:00,https://t.co/6gs25lZoOO Outro grande banco hol...,another great dutch bank abandons portfolio la...,0.800000
2019-05-10 09:59:45+00:00,@mark_dow Bitcoin is the one imv,dow bitcoin one imv,0.000000
2019-05-27 11:49:47+00:00,This is true. Crypto currency is a new asset c...,true crypto currency new asset class people be...,0.243182
2019-05-27 11:50:04+00:00,"Bitcoin dispara, fica muito perto de US$ 9000 ...",bitcoin shoots gets close reaches maximum,0.000000
2019-05-27 11:50:21+00:00,Bitcoin rallies 10% to hit 12-month high https...,bitcoin rallies hit month high,0.160000
2019-05-27 11:49:54+00:00,#LTCprice surges ahead of $BTC quotes as users...,ltcprice surges ahead btc quotes users show in...,0.500000
